In [1]:
import warnings
import os
import logging
import numpy as np
import mindspore
import mindspore.nn as nn
from mindnlp.modules import CRF
from mindspore import Parameter, Tensor
from mindspore.nn import AdamWeightDecay as Adam
from mindnlp.models import BertConfig, BertModel
import mindspore.dataset as ds
from mindspore.dataset.text import Vocab as msVocab
from mindnlp.transforms import BertTokenizer
from tqdm import tqdm_notebook,tqdm
import random

warnings.filterwarnings("ignore")
# 指定显卡
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    mindspore.set_seed(seed)
    mindspore.dataset.config.set_seed(seed)

# 读取文本，返回词典，索引表，句子，标签
def read_data(path):
    sentences = []
    labels = []
    with open(path, 'r', encoding='utf-8') as f:
        sent = []
        label = []
        for line in f:
            parts = line.split()
            if len(parts) == 0:
                if len(sent) != 0:
                    sentences.append(sent)
                    labels.append(label)
                sent = []
                label = []
            else:
                sent.append(parts[0])
                label.append(parts[-1])
                
    return (sentences, labels)

def read_vocab(path):
    vocab_list = []
    with open(path, 'r', encoding='utf-8') as f:
        for word in f:
            vocab_list.append(word.strip())
    return vocab_list

def get_entity(decode):
    starting = False
    p_ans = []
    for i,label in enumerate(decode):
        if label > 0:
            if label%2==1:
                starting=True
                p_ans.append(([i],labels_text_mp[label//2]))
            elif starting:
                p_ans[-1][0].append(i)
        else:
            starting=False
    return p_ans

# 处理数据 
class Feature(object):
    def __init__(self,sent, label):
        self.sent = sent
        label = [LABEL_MAP[c] for c in label]
        self.token_ids = list(tokenizer(' '.join(sent)))
        self.seq_length = len(self.token_ids) if len(self.token_ids) - 2 < Max_Len else Max_Len + 2
        assert self.seq_length <= Max_Len + 2, f"seq_length:{self.seq_length}"
        offset = tokenizer.encode(' '.join(sent)).offsets
        self.labels = self.get_labels(offset, label)
        assert len(self.token_ids) == len(self.labels)+2, f"长度不一致:\ntoken_ids:{len(self.token_ids)}\nlabels:{len(self.labels)}\ntext:{len(sent)}"
        self.labels = [0] + self.labels[:Max_Len] + [0]
        self.labels = self.labels + [0]*(Max_Len - len(self.labels) + 2)
        
        self.token_ids = [101] + self.token_ids[1:-1][:Max_Len] + [102]
        self.token_ids = self.token_ids + [0]*(Max_Len - len(self.token_ids) + 2)
        self.entity = get_entity(self.labels)
        
    def get_labels(self, offset_mapping, label):
        sent_len, count, index = 0, 0, 0
        label_new = []
        for l, r in offset_mapping:
            if l != 0 or r != 0:
                if count == sent_len:
                    sent_len += len(self.sent[index])
                    index += 1
                count += r - l
                label_new.append(label[index-1])
                
        return label_new

class GetDatasetGenerator:
    def __init__(self, data):
        self.features = [Feature(data[0][i], data[1][i]) for i in range(len(data[0]))]
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        feature = self.features[index]
        token_ids = feature.token_ids
        labels = feature.labels
        
        return (token_ids, feature.seq_length, labels)
    
def debug_dataset(dataset):
    dataset = dataset.batch(batch_size=16)
    for data in dataset.create_dict_iterator():
        print(data["data"].shape, data["label"].shape)
        break
        
def get_metric(P_ans, valid):
    predict_score = 0 # 预测正确个数
    predict_number = 0 # 预测结果个数
    totol_number = 0 # 标签个数
    for i in range(len(P_ans)):
        predict_number += len(P_ans[i])
        totol_number += len(valid.features[i].entity)
        pred_true = [x for x in valid.features[i].entity if x in P_ans[i]]
        predict_score += len(pred_true)
    P = predict_score/predict_number if predict_number>0 else 0.
    R = predict_score/totol_number if totol_number>0 else 0.
    f1=(2*P*R)/(P+R) if (P+R)>0 else 0.
    print(f'f1 = {f1}， P(准确率) = {P}, R(召回率) = {R}')
    
def get_optimizer(model):
    param_optimizer = list(model.parameters_and_names())
    
    no_decay = ['bias', 'layer_norm.bias', 'layer_norm.weight']
    crf_p = [n for n, p in param_optimizer if str(n).find('crf') != -1]

    optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and  n not in crf_p], 'weight_decay': 0.8},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and n not in crf_p], 'weight_decay': 0.0},
            {'params': [p for n, p in param_optimizer if n in crf_p], 'lr': 3e-3,'weight_decay': 0.8},
            ]
    optimizer = AdamW(optimizer_grouped_parameters, learning_rate=3e-5, eps=1e-8) # 学习率不宜过大，不然预测结果可能都是0

    return optimizer

In [4]:
class Bert_LSTM_CRF(nn.Cell):
    def __init__(self, num_labels):
        super().__init__()
        self.num_labels = num_labels
        config = BertConfig.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', config=config)
        self.bilstm = nn.LSTM(config.hidden_size, config.hidden_size//2, batch_first=True, bidirectional=True)
        self.crf_hidden_fc = nn.Dense(config.hidden_size, self.num_labels)
        self.crf = CRF(self.num_labels, batch_first=True, reduction='mean')

    def construct(self, ids, seq_length=None, labels=None):
        attention_mask = (ids > 0)
        output = self.bert_model(input_ids=ids, attention_mask=attention_mask)
        lstm_feat, _ = self.bilstm(output[0])
        emissions = self.crf_hidden_fc(lstm_feat)
        loss_crf = self.crf(emissions, tags=labels, seq_length=seq_length)
        
        return loss_crf

In [6]:
seed_everything(42)
root = '../data/en/ner/' # 文件目录
Max_Len = 113
Entity = ['PER', 'LOC', 'ORG', 'MISC']
labels_text_mp={k:v for k,v in enumerate(Entity)}
LABEL_MAP = {'O': 0}
for i, e in enumerate(Entity):
    LABEL_MAP[f'B-{e}'] = 2 * (i+1) - 1
    LABEL_MAP[f'I-{e}'] = 2 * (i+1)

In [7]:
train = read_data(root+'train.txt')
test = read_data(root+'test.txt')
dev = read_data(root+'valid.txt')

In [8]:
Epoch = 3
batch_size = 16
vocab_list = read_vocab('../../Pretrain/bert-base-uncased/vocab.txt')
vocab = msVocab.from_list(vocab_list)
tokenizer = BertTokenizer(vocab=vocab, lower_case=True)
dataset_generator = GetDatasetGenerator(train)
dataset = ds.GeneratorDataset(dataset_generator, ["data", "length", "label"], shuffle=False)
dataset_train = dataset.batch(batch_size=batch_size)

In [9]:
model = Bert_LSTM_CRF(num_labels=len(Entity)*2+1)
optimizer = get_optimizer(model)
grad_fn = mindspore.value_and_grad(model, None, optimizer.parameters)

[WARNING] ME(59298:140049833940800,MainProcess):2023-04-27-12:07:39.629.966 [mindspore/train/serialization.py:1109] For 'load_param_into_net', remove parameter prefix name: bert., continue to load.


In [10]:
# 训练
size = dataset_train.get_dataset_size()
steps = size
tloss = []
for epoch in range(Epoch):
    model.set_train()
    with tqdm(total=steps) as t:
        for batch, (token_ids, seq_length, labels) in enumerate(dataset_train.create_tuple_iterator()):
            loss, grads = grad_fn(token_ids, seq_length, labels)
            optimizer(grads)
            tloss.append(loss.asnumpy())
            t.set_postfix(loss=np.array(tloss).mean())
            t.update(1)
    # mindspore.save_checkpoint(model, f"../model/ms_seed_42_model{epoch}.ckpt")

100%|██████████| 878/878 [13:18<00:00,  1.10it/s, loss=1.59]


In [11]:
# 预测：train
dataset_generator = GetDatasetGenerator(train)
dataset = ds.GeneratorDataset(dataset_generator, ["data", "length", "label"], shuffle=False)
dataset_train = dataset.batch(batch_size=batch_size)

size = dataset_train.get_dataset_size()
steps = size
decodes=[]
model.set_train(False)
with tqdm(total=steps) as t:
    for batch, (token_ids, seq_length, labels) in enumerate(dataset_train.create_tuple_iterator()):
        score, history = model(token_ids, seq_length=seq_length)
        best_tags = model.crf.post_decode(score, history, seq_length)
        decode = [[y.asnumpy().item() for y in x] for x in best_tags]
        decodes.extend(list(decode))
        t.update(1)
        
v_pred = [get_entity(x) for x in decodes]
get_metric(v_pred, dataset_generator)

100%|██████████| 878/878 [05:15<00:00,  2.78it/s]

f1 = 0.9703738472582694， P(准确率) = 0.9681284777698198, R(召回率) = 0.9726296562794045


In [12]:
# 预测：dev
dev_dataset_generator = GetDatasetGenerator(dev)
dataset_dev = ds.GeneratorDataset(dev_dataset_generator, ["data", "length", "label"], shuffle=False)
dataset_dev = dataset_dev.batch(batch_size=batch_size)

size = dataset_dev.get_dataset_size()
steps = size
decodes=[]
model.set_train(False)
with tqdm(total=steps) as t:
    for batch, (token_ids, seq_length, labels) in enumerate(dataset_dev.create_tuple_iterator()):
        score, history = model(token_ids, seq_length=seq_length)#.asnumpy()
        best_tags = model.crf.post_decode(score, history, seq_length)
        decode = [[y.asnumpy().item() for y in x] for x in best_tags]
        decodes.extend(list(decode))
        t.update(1)
v_pred = [get_entity(x) for x in decodes]
get_metric(v_pred, dev_dataset_generator)

100%|██████████| 204/204 [01:14<00:00,  2.75it/s]

f1 = 0.925837453474807， P(准确率) = 0.9179334655210398, R(召回率) = 0.9338787403339684


In [13]:
# 预测：test
test_dataset_generator = GetDatasetGenerator(test)
dataset_test = ds.GeneratorDataset(test_dataset_generator, ["data", "length", "label"], shuffle=False)
dataset_test = dataset_test.batch(batch_size=batch_size)

size = dataset_test.get_dataset_size()
steps = size
decodes_pred=[]
model.set_train(False)
with tqdm(total=steps) as t:
    for batch, (token_ids, seq_length, labels) in enumerate(dataset_test.create_tuple_iterator()):
        score, history = model(token_ids, seq_length=seq_length)
        best_tags = model.crf.post_decode(score, history, seq_length)
        decode = [[y.asnumpy().item() for y in x] for x in best_tags]
        decodes_pred.extend(list(decode))
        t.update(1)
        

pred = [get_entity(x) for x in decodes_pred]
get_metric(pred, test_dataset_generator)

100%|██████████| 216/216 [01:16<00:00,  2.82it/s]

f1 = 0.879166175951007， P(准确率) = 0.8718757299696333, R(召回率) = 0.8865795724465558
